# Анализ эксперимента neural_controller-v1

## Постановка эксперимента

**Задача**: стабилизация лазера с помощью нейросетевого контроллера, обученного методом Twin Delayed DDPG (TD3).

**Окружение**: `NeuralPIDDeltaEnv` — агент управляет фазовращателем через последовательный порт (COM3). 
- **Наблюдение**: `[error, error_prev, error_prev_prev]` — текущая и две предыдущие ошибки (нормализованы).
- **Действие**: нормализованное приращение управляющего сигнала `delta_norm ∈ [-1, 1]`, масштабируемое до `delta ∈ [-30, 30]`.
- **Награда**: `-|error|` — минимизация абсолютной ошибки.
- **Уставка (setpoint)**: 1200 (относительно `process_variable_max = 10230`).
- **Управляющий сигнал**: `control_output ∈ [0, 4095]`, начальное значение при ресете — 2000.

**Алгоритм**: TD3, MLP 256×256, `gamma = 0.99`, `tau = 0.005`, `policy_freq = 2`.

**Сбор данных**: асинхронный коллектор, синхронизация весов каждые 10 шагов.

**Исследование (exploration)**: PID-контроллер (`kp=3.5, ki=11.0, kd=0.002`) в течение первых 20 000 шагов окружения.

**Обучение**: начинается после 5 000 шагов буфера, буфер на 100 000 переходов, батч 256.

**Длительность**: 2026-02-13 14:59 → 16:13 (~1 ч 14 мин), прерван пользователем.

In [ ]:
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from pathlib import Path
from nn_laser_stabilizer.config.config import load_config
from nn_laser_stabilizer.paths import get_experiment_dir

In [ ]:
EXPERIMENT_NAME = "neural_controller-v1"
EXPERIMENT_DATE = "2026-02-13"
EXPERIMENT_TIME = "14-59-23"

EXPERIMENT_DIR_PATH = get_experiment_dir(
    experiment_name=EXPERIMENT_NAME, 
    experiment_date=EXPERIMENT_DATE, 
    experiment_time=EXPERIMENT_TIME)

config = load_config(EXPERIMENT_DIR_PATH / "config.yaml")
print(f"Эксперимент: {config.experiment_name}")
print(f"Окружение: {config.env.name}")
print(f"Алгоритм: {config.algorithm.type}")
print(f"Exploration steps: {config.exploration.steps}")
print(f"Train start step: {config.training.train_start_step}")

## Загрузка и парсинг данных

In [ ]:
def load_jsonl(path: Path, source: str | None = None, event: str | None = None) -> pd.DataFrame:
    rows = []
    with open(path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                record = json.loads(line)
            except json.JSONDecodeError:
                continue
            if source and record.get("source") != source:
                continue
            if event and record.get("event") != event:
                continue
            rows.append(record)
    return pd.DataFrame(rows)

In [ ]:
env_df = load_jsonl(EXPERIMENT_DIR_PATH / "env.jsonl", source="env", event="step")
env_df = env_df.reset_index(drop=True)
env_df['global_step'] = env_df.index + 1  # абсолютный номер шага (step сбрасывается при reset)
print(f"Шаги окружения: {len(env_df)} записей")
print(f"Диапазон global_step: {env_df['global_step'].min()} — {env_df['global_step'].max()}")
env_df.tail()

In [ ]:
train_df = load_jsonl(EXPERIMENT_DIR_PATH / "train.jsonl", source="train", event="step")
print(f"Шаги обучения: {len(train_df)} записей")
print(f"Диапазон шагов: {train_df['step'].min()} — {train_df['step'].max()}")
train_df.head()

In [ ]:
exploration_steps = config.exploration.steps
print(f"Exploration steps: {exploration_steps}")

## Анализ логов окружения

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['error'], alpha=0.8, linewidth=0.5, label='Error')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Ошибка регулирования')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Error')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['reward'], alpha=0.8, linewidth=0.5, color='tab:green', label='Reward')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Награда')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Reward')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['process_variable'], 'b-', alpha=0.7, linewidth=0.5, label='Process Variable')
plt.plot(env_df['global_step'], env_df['setpoint'], 'r--', linewidth=1.5, label='Setpoint')
plt.axvline(x=exploration_steps, color='orange', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Значение')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['control_output'], 'g-', alpha=0.7, linewidth=0.5, label='Control Output')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Управляющий сигнал')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Control Output')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['delta_norm'], alpha=0.7, linewidth=0.5, color='tab:purple', label='Delta (norm)')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Действие агента (нормализованное приращение)')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('delta_norm')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['step_interval_us'], alpha=0.7, linewidth=0.5, color='tab:brown')
plt.axvline(x=exploration_steps, color='red', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Интервал между шагами')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('Интервал (мкс)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Сравнение PID-exploration vs НС-контроллер

In [ ]:
pid_df = env_df[env_df['global_step'] <= exploration_steps]
nn_df = env_df[env_df['global_step'] > exploration_steps]

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(pid_df['error'], bins=100, alpha=0.7, color='tab:blue', edgecolor='black', linewidth=0.3)
axes[0].set_title(f'Распределение ошибки: PID (шаги 1–{exploration_steps})')
axes[0].set_xlabel('Error')
axes[0].set_ylabel('Частота')
axes[0].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[0].grid(True, alpha=0.3)

axes[1].hist(nn_df['error'], bins=100, alpha=0.7, color='tab:orange', edgecolor='black', linewidth=0.3)
axes[1].set_title(f'Распределение ошибки: НС (шаги {exploration_steps + 1}–{env_df["global_step"].max()})')
axes[1].set_xlabel('Error')
axes[1].set_ylabel('Частота')
axes[1].axvline(x=0, color='black', linestyle='-', linewidth=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"PID  — mean |error|: {pid_df['error'].abs().mean():.2f}, std: {pid_df['error'].std():.2f}, median: {pid_df['error'].median():.2f}")
print(f"НС   — mean |error|: {nn_df['error'].abs().mean():.2f}, std: {nn_df['error'].std():.2f}, median: {nn_df['error'].median():.2f}")

## Скользящее среднее ошибки

In [ ]:
alpha = 0.2
env_df['abs_error_ema'] = env_df['error'].abs().ewm(alpha=alpha, adjust=False).mean()

plt.figure(figsize=(14, 5))
plt.plot(env_df['global_step'], env_df['abs_error_ema'], linewidth=1.0, color='tab:red',
         label=f'|Error| (EMA, α={alpha})')
plt.axvline(x=exploration_steps, color='gray', linestyle='--', linewidth=2,
            label=f'Переход на НС (шаг {exploration_steps})')
plt.title('Экспоненциальное скользящее среднее абсолютной ошибки')
plt.xlabel('Шаг окружения (абсолютный)')
plt.ylabel('|Error| (EMA)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Анализ процесса обучения

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['loss_q1'], alpha=0.6, linewidth=0.3, color='tab:blue', label='Q1 Loss')
plt.title('Critic Q1 Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['loss_q2'], alpha=0.6, linewidth=0.3, color='tab:green', label='Q2 Loss')
plt.title('Critic Q2 Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
actor_df = train_df[train_df['actor_loss'].notna()]

plt.figure(figsize=(14, 5))
plt.plot(actor_df['step'], actor_df['actor_loss'], alpha=0.6, linewidth=0.3, color='tab:red', label='Actor Loss')
plt.title('Actor Loss')
plt.xlabel('Шаг обучения')
plt.ylabel('Loss')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(train_df['step'], train_df['buffer_size'], linewidth=1.0, color='tab:purple')
plt.title('Размер буфера воспроизведения')
plt.xlabel('Шаг обучения')
plt.ylabel('Размер буфера')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Анализ логов соединения (phase_shifter)

Логи `source=phase_shifter, event=exchange` — сырые обмены с фазовращателем через последовательный порт. Каждая запись содержит отправленный `control_output` и полученный `process_variable`. Эти данные включают все обмены, в том числе во время reset-фаз окружения.

In [ ]:
ps_df = load_jsonl(EXPERIMENT_DIR_PATH / "env.jsonl", source="phase_shifter", event="exchange")
ps_df = ps_df.reset_index(drop=True)
ps_df['global_step'] = ps_df.index + 1
print(f"Обмены с фазовращателем: {len(ps_df)} записей")
print(f"control_output: [{ps_df['control_output'].min()}, {ps_df['control_output'].max()}]")
print(f"process_variable: [{ps_df['process_variable'].min()}, {ps_df['process_variable'].max()}]")
ps_df.head()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(ps_df['global_step'], ps_df['process_variable'], alpha=0.7, linewidth=0.3, color='tab:blue')
plt.title('Process Variable (логи соединения)')
plt.xlabel('Номер обмена')
plt.ylabel('Process Variable')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(14, 5))
plt.plot(ps_df['global_step'], ps_df['control_output'], alpha=0.7, linewidth=0.3, color='tab:green')
plt.title('Control Output (логи соединения)')
plt.xlabel('Номер обмена')
plt.ylabel('Control Output')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(2, 1, figsize=(14, 8), sharex=True)

axes[0].plot(ps_df['global_step'], ps_df['control_output'], alpha=0.7, linewidth=0.3, color='tab:green')
axes[0].set_title('Control Output')
axes[0].set_ylabel('Control Output')
axes[0].grid(True, alpha=0.3)

axes[1].plot(ps_df['global_step'], ps_df['process_variable'], alpha=0.7, linewidth=0.3, color='tab:blue')
axes[1].set_title('Process Variable')
axes[1].set_xlabel('Номер обмена')
axes[1].set_ylabel('Process Variable')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Логи соединения: Control Output и Process Variable', fontsize=13)
plt.tight_layout()
plt.show()

## Анализ моделей

Загрузка сохранённых моделей: actor, critic1, critic2 и их target-копии.

In [ ]:
import torch
from nn_laser_stabilizer.rl.model.actor import MLPActor
from nn_laser_stabilizer.rl.model.critic import MLPCritic

MODELS_DIR = EXPERIMENT_DIR_PATH / "models"

actor = MLPActor.load(MODELS_DIR / "actor.pth").eval()
actor_target = MLPActor.load(MODELS_DIR / "actor_target.pth").eval()
critic1 = MLPCritic.load(MODELS_DIR / "critic1.pth").eval()
critic2 = MLPCritic.load(MODELS_DIR / "critic2.pth").eval()
critic1_target = MLPCritic.load(MODELS_DIR / "critic1_target.pth").eval()
critic2_target = MLPCritic.load(MODELS_DIR / "critic2_target.pth").eval()

print("Actor:")
print(actor)
print(f"\nВсего параметров actor: {sum(p.numel() for p in actor.parameters()):,}")
print(f"Всего параметров critic1: {sum(p.numel() for p in critic1.parameters()):,}")

In [ ]:
def plot_weight_histograms(model: torch.nn.Module, title: str):
    """Гистограммы весов и bias для каждого линейного слоя."""
    linear_layers = [(name, module) for name, module in model.named_modules()
                     if isinstance(module, torch.nn.Linear)]
    n = len(linear_layers)
    fig, axes = plt.subplots(n, 2, figsize=(14, 3 * n))
    if n == 1:
        axes = axes.reshape(1, -1)
    
    for i, (name, layer) in enumerate(linear_layers):
        w = layer.weight.detach().cpu().numpy().flatten()
        axes[i, 0].hist(w, bins=80, alpha=0.7, color='tab:blue', edgecolor='black', linewidth=0.3)
        axes[i, 0].set_title(f'{name} weights [{layer.weight.shape[1]}→{layer.weight.shape[0]}]')
        axes[i, 0].set_ylabel('Частота')
        axes[i, 0].grid(True, alpha=0.3)
        
        b = layer.bias.detach().cpu().numpy().flatten()
        axes[i, 1].hist(b, bins=40, alpha=0.7, color='tab:orange', edgecolor='black', linewidth=0.3)
        axes[i, 1].set_title(f'{name} bias [{layer.bias.shape[0]}]')
        axes[i, 1].grid(True, alpha=0.3)
    
    plt.suptitle(title, fontsize=14)
    plt.tight_layout()
    plt.show()

In [ ]:
plot_weight_histograms(actor, 'Распределение весов: Actor')

In [ ]:
plot_weight_histograms(critic1, 'Распределение весов: Critic 1')

### Визуализация политики: heatmap action(error, error_prev)

Подаём на вход актору сетку значений `(error, error_prev)` при фиксированном `error_prev_prev = 0` и строим heatmap выходного действия. Значения ошибок нормализованы в `[-1, 1]`.

In [ ]:
grid_n = 200
error_range = np.linspace(-1, 1, grid_n)
error_prev_range = np.linspace(-1, 1, grid_n)
error_grid, error_prev_grid = np.meshgrid(error_range, error_prev_range)

# error_prev_prev = 0 (фиксируем)
obs_grid = np.stack([
    error_grid.flatten(),
    error_prev_grid.flatten(),
    np.zeros(grid_n * grid_n),
], axis=1)

obs_tensor = torch.tensor(obs_grid, dtype=torch.float32)

with torch.no_grad():
    actions, _ = actor(obs_tensor)
    actions_np = actions.cpu().numpy().reshape(grid_n, grid_n)

plt.figure(figsize=(10, 8))
im = plt.imshow(actions_np, extent=[-1, 1, -1, 1], origin='lower', aspect='auto', cmap='RdBu_r')
plt.colorbar(im, label='action (delta_norm)')
plt.xlabel('error (norm)')
plt.ylabel('error_prev (norm)')
plt.title('Политика актора: action(error, error_prev) при error_prev_prev=0')
plt.grid(False)
plt.tight_layout()
plt.show()

In [ ]:
# Срез: action(error) при error_prev=0, error_prev_prev=0
obs_1d = torch.tensor(
    np.stack([error_range, np.zeros(grid_n), np.zeros(grid_n)], axis=1),
    dtype=torch.float32,
)

with torch.no_grad():
    actions_1d, _ = actor(obs_1d)
    actions_1d_np = actions_1d.cpu().numpy().flatten()

plt.figure(figsize=(10, 5))
plt.plot(error_range, actions_1d_np, linewidth=2, color='tab:red')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.xlabel('error (norm)')
plt.ylabel('action (delta_norm)')
plt.title('Срез политики: action(error) при error_prev=0, error_prev_prev=0')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### Сравнение actor и actor_target

L2-расстояние между параметрами основной и целевой сети, а также разница в действиях на одних и тех же входах.

In [ ]:
# L2-расстояние между параметрами actor и actor_target
l2_distances = {}
for (name, p), (_, p_target) in zip(actor.named_parameters(), actor_target.named_parameters()):
    l2 = (p - p_target).norm().item()
    l2_distances[name] = l2

print("L2-расстояние между actor и actor_target по слоям:")
for name, dist in l2_distances.items():
    print(f"  {name}: {dist:.6f}")

total_l2 = sum(
    (p - p_t).pow(2).sum().item()
    for p, p_t in zip(actor.parameters(), actor_target.parameters())
) ** 0.5
print(f"\nОбщее L2-расстояние: {total_l2:.6f}")

In [ ]:
# Разница действий actor vs actor_target на сетке
with torch.no_grad():
    actions_target, _ = actor_target(obs_tensor)
    actions_target_np = actions_target.cpu().numpy().reshape(grid_n, grid_n)

diff_np = actions_np - actions_target_np

plt.figure(figsize=(10, 8))
vmax = max(abs(diff_np.min()), abs(diff_np.max()))
im = plt.imshow(diff_np, extent=[-1, 1, -1, 1], origin='lower', aspect='auto',
                cmap='RdBu_r', vmin=-vmax, vmax=vmax)
plt.colorbar(im, label='action - action_target')
plt.xlabel('error (norm)')
plt.ylabel('error_prev (norm)')
plt.title('Разница политик: actor − actor_target (error_prev_prev=0)')
plt.grid(False)
plt.tight_layout()
plt.show()

## Анализ буфера воспроизведения

Загрузка сохранённого `ReplayBuffer` и анализ распределений наблюдений, действий и наград.

In [ ]:
from nn_laser_stabilizer.rl.data.replay_buffer import ReplayBuffer

buffer = ReplayBuffer.load(EXPERIMENT_DIR_PATH / "data" / "replay_buffer.pth")
n = buffer.size
print(f"Размер буфера: {n}")
print(f"Ёмкость: {buffer.capacity}")
print(f"obs_dim: {buffer.observations.shape[1]}, action_dim: {buffer.actions.shape[1]}")

buf_obs = buffer.observations[:n].numpy()       # (N, 3): error, error_prev, error_prev_prev
buf_actions = buffer.actions[:n].numpy()         # (N, 1): delta_norm
buf_rewards = buffer.rewards[:n].numpy()         # (N, 1)
buf_next_obs = buffer.next_observations[:n].numpy()
buf_dones = buffer.dones[:n].numpy()

print(f"\nСтатистика наблюдений (нормализованные):")
for i, name in enumerate(['error', 'error_prev', 'error_prev_prev']):
    print(f"  {name}: mean={buf_obs[:, i].mean():.4f}, std={buf_obs[:, i].std():.4f}, "
          f"min={buf_obs[:, i].min():.4f}, max={buf_obs[:, i].max():.4f}")

print(f"\nСтатистика действий:")
print(f"  delta_norm: mean={buf_actions.mean():.4f}, std={buf_actions.std():.4f}, "
      f"min={buf_actions.min():.4f}, max={buf_actions.max():.4f}")

print(f"\nСтатистика наград:")
print(f"  reward: mean={buf_rewards.mean():.4f}, std={buf_rewards.std():.4f}, "
      f"min={buf_rewards.min():.4f}, max={buf_rewards.max():.4f}")

print(f"\nDones: {buf_dones.sum()} / {n} ({buf_dones.mean() * 100:.2f}%)")

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

axes[0, 0].hist(buf_obs[:, 0], bins=100, alpha=0.7, color='tab:blue', edgecolor='black', linewidth=0.3)
axes[0, 0].set_title('error (norm)')
axes[0, 0].set_ylabel('Частота')
axes[0, 0].grid(True, alpha=0.3)

axes[0, 1].hist(buf_obs[:, 1], bins=100, alpha=0.7, color='tab:cyan', edgecolor='black', linewidth=0.3)
axes[0, 1].set_title('error_prev (norm)')
axes[0, 1].grid(True, alpha=0.3)

axes[1, 0].hist(buf_actions.flatten(), bins=100, alpha=0.7, color='tab:purple', edgecolor='black', linewidth=0.3)
axes[1, 0].set_title('action (delta_norm)')
axes[1, 0].set_xlabel('Значение')
axes[1, 0].set_ylabel('Частота')
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(buf_rewards.flatten(), bins=100, alpha=0.7, color='tab:green', edgecolor='black', linewidth=0.3)
axes[1, 1].set_title('reward')
axes[1, 1].set_xlabel('Значение')
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Распределения данных в буфере воспроизведения', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.scatter(buf_obs[:, 0], buf_actions.flatten(), alpha=0.05, s=1, color='tab:purple')
plt.xlabel('error (norm)')
plt.ylabel('action (delta_norm)')
plt.title('Буфер: зависимость action от error')
plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
plt.hist2d(buf_obs[:, 0], buf_obs[:, 1], bins=100, cmap='hot_r')
plt.colorbar(label='Количество переходов')
plt.xlabel('error (norm)')
plt.ylabel('error_prev (norm)')
plt.title('Покрытие пространства состояний в буфере')
plt.tight_layout()
plt.show()

## Оценка Q-функции

Оценка Q(s, a) на данных из буфера: как критик оценивает качество действий, накопленных в буфере. Также — Q-значение для действий, предложенных актором (Q(s, π(s))).

In [ ]:
obs_t = torch.tensor(buf_obs, dtype=torch.float32)
act_t = torch.tensor(buf_actions, dtype=torch.float32)

with torch.no_grad():
    # Q(s, a) — оценка действий из буфера
    q1_buffer, _ = critic1(obs_t, act_t)
    q2_buffer, _ = critic2(obs_t, act_t)
    q_min_buffer = torch.min(q1_buffer, q2_buffer).numpy().flatten()
    
    # Q(s, π(s)) — оценка действий актора
    actor_actions, _ = actor(obs_t)
    q1_policy, _ = critic1(obs_t, actor_actions)
    q2_policy, _ = critic2(obs_t, actor_actions)
    q_min_policy = torch.min(q1_policy, q2_policy).numpy().flatten()

print(f"Q(s, a_buffer): mean={q_min_buffer.mean():.4f}, std={q_min_buffer.std():.4f}, "
      f"min={q_min_buffer.min():.4f}, max={q_min_buffer.max():.4f}")
print(f"Q(s, π(s)):     mean={q_min_policy.mean():.4f}, std={q_min_policy.std():.4f}, "
      f"min={q_min_policy.min():.4f}, max={q_min_policy.max():.4f}")

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

axes[0].hist(q_min_buffer, bins=100, alpha=0.7, color='tab:blue', edgecolor='black', linewidth=0.3)
axes[0].set_title('Q(s, a) — действия из буфера')
axes[0].set_xlabel('Q-value')
axes[0].set_ylabel('Частота')
axes[0].grid(True, alpha=0.3)

axes[1].hist(q_min_policy, bins=100, alpha=0.7, color='tab:red', edgecolor='black', linewidth=0.3)
axes[1].set_title('Q(s, π(s)) — действия актора')
axes[1].set_xlabel('Q-value')
axes[1].grid(True, alpha=0.3)

plt.suptitle('Распределение Q-значений (min(Q1, Q2))', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
# Q(s, π(s)) как функция error
plt.figure(figsize=(10, 5))
plt.scatter(buf_obs[:, 0], q_min_policy, alpha=0.05, s=1, color='tab:red')
plt.xlabel('error (norm)')
plt.ylabel('Q(s, π(s))')
plt.title('Q-значение политики актора в зависимости от ошибки')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5, alpha=0.5)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Heatmap Q(s, π(s)) на сетке error × error_prev
with torch.no_grad():
    grid_actions, _ = actor(obs_tensor)
    q1_grid, _ = critic1(obs_tensor, grid_actions)
    q2_grid, _ = critic2(obs_tensor, grid_actions)
    q_grid = torch.min(q1_grid, q2_grid).numpy().reshape(grid_n, grid_n)

plt.figure(figsize=(10, 8))
im = plt.imshow(q_grid, extent=[-1, 1, -1, 1], origin='lower', aspect='auto', cmap='viridis')
plt.colorbar(im, label='Q(s, π(s))')
plt.xlabel('error (norm)')
plt.ylabel('error_prev (norm)')
plt.title('Q-значение политики: Q(s, π(s)) при error_prev_prev=0')
plt.grid(False)
plt.tight_layout()
plt.show()